In [ ]:
!pip install -q flask-ngrok
!pip install -q pyngrok

In [ ]:
from pyngrok import ngrok

In [ ]:
from google.colab import userdata
key = userdata.get('grok_token')

In [ ]:
!ngrok authtoken {key}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import requests


In [ ]:
ngrok.kill()

public_url = ngrok.connect(8080)
print(public_url)

NgrokTunnel: "https://e240-34-27-254-176.ngrok-free.app" -> "http://localhost:8080"


In [ ]:
!mkdir templates

In [ ]:
%%writefile templates/base.html
<!DOCTYPE html>
<html>
<head>
    <title>LangAlpha</title>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    <style>
        body {
            font-family: sans-serif;
        }
        .navbar {
            background-color: #333;
            overflow: hidden;
        }
        .navbar a {
            float: left;
            display: block;
            color: white;
            text-align: center;
            padding: 14px 16px;
            text-decoration: none;
        }
        .navbar a:hover {
            background-color: #ddd;
            color: black;
        }
        .search-container {
            text-align: center;
            margin-top: 50px;
        }
        .search-container input[type="text"] {
            padding: 10px;
            border: 1px solid #ccc;
            border-radius: 5px;
            width: 300px;
        }
        .search-container button {
            padding: 10px 20px;
            background-color: #007bff;
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }
    </style>
</head>
<body>
    <div class="navbar">
        <a href="/">Home</a>
        <a href="/about">About Us</a>
        <a href="/analysis">Analysis</a>
        <a href="/ai">AI Chat</a>
    </div>
    {% block content %}{% endblock %}
</body>
</html>

Writing templates/base.html


In [ ]:
%%writefile templates/title.html
{% extends "base.html" %}
{% block content %}

<div class="container search-container">
    <h1>Welcome to LangAlpha</h1>
    <h5>Put a ticker for analysis</h5>
    <form action="/search">
        <input type="text" name="tic" placeholder="Enter Stock Ticker" required>
        <button type="submit">Search</button>
    </form>
</div>

{% endblock %}

Writing templates/title.html


In [ ]:
%%writefile templates/ABOUT.html
{% extends "base.html" %}
{% block content %}

<div class="container">
  <div class="row">
    <div class="col-md-12 text-center" style="padding-top: 100px;">
      <h1>Thanks for using our website!</h1>
      <h5>We are students from NYU working on this project for a class. We hope you enjoy using it!</h5>
      <img class="center-block" src="{{ image_url }}" />
    </div>
  </div>
</div>

{% endblock %}

Writing templates/ABOUT.html


In [ ]:
#need to fix this, required is not save
#not all are number too
%%writefile templates/ANALYSIS.html
{% extends "base.html" %}
{% block content %}

<div class="container">
    <div class="row">
        <div class="col-md-12" style="padding-top: 10px;">
            <h1>Stock Analysis</h1>
            <form action="/search" method="GET">
                <div class="form-group">
                    <label for="tic">Enter Stock Ticker:</label>
                    <input type="text" class="form-control" id="tic" name="tic" placeholder="e.g., AAPL" required>
                </div>
                <div class="form-group">
                    <label for="number2">Number 2:</label>
                    <input type="number" class="form-control" id="number2" name="number2" placeholder="Enter a number" required>
                </div>
                <div class="form-group">
                    <label for="number3">Number 3:</label>
                    <input type="number" class="form-control" id="number3" name="number3" placeholder="Enter a number" required>
                </div>
                <div class="form-group">
                    <label for="number4">Number 4:</label>
                    <input type="number" class="form-control" id="number4" name="number4" placeholder="Enter a number" required>
                </div>
                <div class="form-group">
                    <label for="number5">Number 5:</label>
                    <input type="number" class="form-control" id="number5" name="number5" placeholder="Enter a number" required>
                </div>
                <div class="form-group">
                    <label for="number6">Number 6:</label>
                    <input type="number" class="form-control" id="number6" name="number6" placeholder="Enter a number" required>
                </div>
                <div class="form-group">
                    <label for="number7">Number 7:</label>
                    <input type="number" class="form-control" id="number7" name="number7" placeholder="Enter a number" required>
                </div>
                <button type="submit" class="btn btn-primary">Submit</button>
            </form>
        </div>
    </div>
</div>

{% endblock %}

Writing templates/ANALYSIS.html


In [ ]:
!mkdir static

In [16]:
import base64

# Load the image (replace 'static/your_image.png' with the actual path)
with open("static/new-york-university.jpg", "rb") as img_file:
    encoded_string = base64.b64encode(img_file.read()).decode('utf-8')

In [17]:
%%writefile templates/Chat.html
{% extends "base.html" %}
{% block content %}
<!DOCTYPE html>
<html>
<head>
  <title>AI Chat</title>
  <style>
    body {
      font-family: sans-serif;
      margin: 0; /* Remove default body margins */
    }

    /* Center the chat container */
    .chat-container {
      width: 80%;
      height: 80vh;
      display: flex;
      flex-direction: column;
      border: 1px solid #ccc; /* Add a border for visual distinction */
      padding: 20px; /* Add some padding within the container */
      position: absolute; /* Position absolutely within the body */
      top: 50%;
      left: 50%;
      transform: translate(-50%, -50%); /* Center using translate */
    }

    /* Style for chat messages */
    .chat-messages {
      flex: 1; /* Allow chat messages to take up available space */
      overflow-y: auto; /* Add vertical scrollbar if needed */
      padding: 10px;
    }

    /* Style for individual messages */
    .chat-messages div {
      margin-bottom: 10px;
    }

    /* Style for user messages */
    .chat-messages div:nth-child(odd) { /* User messages are odd-numbered */
      text-align: right;
    }

    /* Style for AI messages */
    .chat-messages div:nth-child(even) { /* AI messages are even-numbered */
      text-align: left;
    }

    /* Style for chat input area */
    .chat-input {
      display: flex;
      padding: 10px;
    }

    /* Style for input field */
    .chat-input input[type="text"] {
      flex: 1; /* Allow input field to take up available space */
      padding: 10px;
      border: 1px solid #ccc;
      border-radius: 5px;
      margin-right: 10px;
    }

    /* Style for send button */
    .chat-input button {
      padding: 10px 20px;
      background-color: #007bff;
      color: white;
      border: none;
      border-radius: 5px;
      cursor: pointer;
    }
  </style>
</head>
<body>
  <div class="chat-container">
    <div class="chat-messages" id="chat-messages">
      </div>
    <div class="chat-input">
      <input type="text" id="user-input" placeholder="Type your message..." onkeyup="handleKeyUp(event)">
      <button onclick="sendMessage()">Send</button>
    </div>
  </div>

  <script>
    function sendMessage() {
      var userInput = document.getElementById("user-input").value;
      var chatMessages = document.getElementById("chat-messages");

      if (userInput.trim() !== "") { // Check if input is not empty
        // Create a new message element for the user's message
        var userMessageElement = document.createElement("div");
        userMessageElement.textContent = "You: " + userInput;
        chatMessages.appendChild(userMessageElement);

        // Simulate a response (replace with actual AI response later)
        var response = "AI: This is a placeholder response.";
        var aiMessageElement = document.createElement("div");
        aiMessageElement.textContent = response;
        chatMessages.appendChild(aiMessageElement);

        // Clear the input field
        document.getElementById("user-input").value = "";

        // Scroll to the bottom of the chat messages
        chatMessages.scrollTop = chatMessages.scrollHeight;
      }
    }

    function handleKeyUp(event) {
      if (event.keyCode === 13) { // Check if Enter key is pressed
        sendMessage();
      }
    }
  </script>
</body>
</html>
{% endblock %}

Writing templates/Chat.html


In [ ]:
from pyngrok import ngrok
# Setup a tunnel to the flask port 5000
ngrok.kill()

public_url = ngrok.connect(5000)
print(public_url)

from flask import Flask, render_template, request, url_for
from flask_ngrok import run_with_ngrok
import requests

app = Flask(__name__,
            template_folder = '/content/templates',
            static_folder='/content/static')
#run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/")
def home():
    return render_template("title.html")

@app.route("/about")
def About():
  image_url = url_for('static', filename='new-york-university.jpg')
  return render_template("ABOUT.html", image_url=image_url)

@app.route("/analysis")
def Analysis():
  return render_template("ANALYSIS.html")

@app.route("/ai")
def Ai():
  return render_template("Chat.html")

app.run()

NgrokTunnel: "https://2542-34-27-254-176.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/Apr/2025 13:24:15] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Apr/2025 13:24:17] "GET /about HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Apr/2025 13:24:17] "GET /static/new-york-university.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Apr/2025 13:24:19] "GET /analysis HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Apr/2025 13:24:20] "GET /ai HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Apr/2025 13:24:29] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Apr/2025 13:24:30] "GET /ai HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Apr/2025 13:24:31] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Apr/2025 13:25:12] "GET /about HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Apr/2025 13:25:12] "GET /static/new-york-unive